In [70]:
import pandas as pd    #pandas dataframe
import re as           # regex expression

#Reading the data
df = pd.read_excel(r"D:\END TO END PROJECTS - GITHUB\PAN Number Data Cleaning & Validation Project in Python\PAN+Card+Validation+in+PYTHON+-+Scripts\PAN Card Validation in PYTHON - Scripts\PAN Number Validation Dataset.xlsx")
#print(df.head(20))

#print('Total records = ',len(df))
#total_records = len(df)     # Assigning length of the dataset to total_records variable

# Data Cleaning
#---------------------------------------------------------------
# Convert PAN column to string dtype and apply string functions
df['Pan_Numbers'] = (
    df['Pan_Numbers']
    .astype('string')       # ensure column is string dtype
    .str.strip()            # remove leading/trailing spaces
    .str.upper()            # convert to uppercase
)
#print(df.head(20))
#df[df['Pan_Numbers'] == ''] #Selects rows where Pan_Numbers is an empty string ('').
#df[df['Pan_Numbers'].isna()] #The isna() function used to detect missing values in a DataFrame or Series

# To check both together (empty string OR NA):
#df = df[df['Pan_Numbers'].isna() | (df['Pan_Numbers'] == '')]

# To replace both together (empty string OR NA):
df = df.replace({"Pan_Numbers":''}, pd.NA).dropna(subset="Pan_Numbers")
#print(df[df['Pan_Numbers']==''])
#print(df[df['Pan_Numbers'].isna()])

#print('Total records = ',len(df)) #Total records =  9033

# To check Unique Values:
# df.nunique() → returns the count of unique values for each column in the dataframe.
# df['Pan_Numbers'] → selects just the Pan_Numbers column.

#print('Total Unique records = ',df.nunique())                  #Total Unique records =  Pan_Numbers    9025
#print('Total Unique records = ',df['Pan_Numbers'].nunique())    #Total Unique records =  9025

#df = df.drop_duplicates()
#print('Total records = ',len(df))
df = df.drop_duplicates(subset="Pan_Numbers", keep='first')
#print('Total records = ',len(df))

# Data Validation
#---------------------------------------------------------------
def has_adjacent_repetition(pan): # AABCD, ABCDX
     for i in range(len(pan)-1):
         if pan[i] == pan[i+1]:
             return True 
     return False 

#print(has_adjacent_repetition("ABCDX"))   # False


def is_sequencial(pan): #ABCDE , ACFGT
     for i in range(len(pan)-1):
         if ord(pan[i+1]) - ord(pan[i]) != 1:
             return False 
     return True 
# ord(char) → gives the Unicode code point (ASCII value) of the character. - ord('A') = 65, ord('B') = 66, etc.
# print(is_sequencial('ABCDE'))
def is_valid_pan(pan):
#length check
    if len(pan) != 10:
        return False 
#regex:#Pattern:#First 5 chars must be uppercase letters [A-Z] - #Next 4 chars must be digits [0-9] - #Last 1 char must be uppercase letter
    if not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$', pan):
        return False 
#repetition check
    if has_adjacent_repetition(pan):
        return False 
#sequential check
    if is_sequencial(pan):
        return False
    return True
#print(is_valid_pan('ZMVA03025O')) # False

df["Status"] = df["Pan_Numbers"].apply(lambda x: "Valid" if is_valid_pan(x) else "Invalid")

valid_cnt = (df["Status"]=='Valid').sum()
invalid_cnt = (df["Status"]=='Invalid').sum()
missing_cnt = total_records - (valid_cnt+invalid_cnt)

print('Total records = ', total_records)
print('Valid = ', valid_cnt)
print('Invalid = ', invalid_cnt)
print('Missing = ', missing_cnt)

df_summary = pd.DataFrame({ "TOTAL PROCESSED RECORDS":[total_records]
                           ,"TOTAL VALID COUNT": [valid_cnt]
                           ,"TOTAL INVALID COUNT": [invalid_cnt]
                           ,"TOTAL MISSING PANS": [missing_cnt]})
#print(df_summary.head())

with pd.ExcelWriter("PAN VALIDATION RESULT - 1.xlsx") as writer:
    df.to_excel(writer, sheet_name="PAN Validations",index=False) # Here we will have Only Pan_Numbers & Status
    df_summary.to_excel(writer, sheet_name="SUMMARY",index=False) # Here we will have Summary columns from df_summary

Total records =  10000
Valid =  3193
Invalid =  5832
Missing =  975
   TOTAL PROCESSED RECORDS  TOTAL VALID COUNT  TOTAL INVALID COUNT  \
0                    10000               3193                 5832   

   TOTAL MISSING PANS  
0                 975  
